In [ ]:
import pandas as pd
from src.algorithm_functions import *

In [21]:
train_df = pd.read_csv('../Datasets/train_df.csv')
train_df.drop('userId', axis=1, inplace=True)

In [22]:
a = fillna_means_combined(train_df, 1/2, 1/2)
a

In [24]:
a